In [16]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from google.cloud import storage
import os

In [17]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "development-416403-b306695feb67.json"

In [18]:
# open file and 
start_time = time.time()
lemmatizer = WordNetLemmatizer()
files = open('llm.txt', 'r')

text = files.read()

# chunk to word
words = [word.lower() for word in text.split() if word.isalpha()]

# remove the stopwords
stop_words = set(stopwords.words('english'))

without_stop_word_file = [word for word in words if word not in stop_words]


#  store the most commom word in list
most_common_word_list = []

for most_common_word, _ in FreqDist(without_stop_word_file).most_common(10):
    most_common_word_list.append(most_common_word)

    # vectorize the list of the most common word
tfid = TfidfVectorizer()
text_vector =tfid.fit_transform(most_common_word_list)
# print(tfid.get_feature_names_out())
# print(text_vector.todense())

# time taken for execution
end_time = time.time()
elapsed_time = end_time - start_time
print("time taken:",elapsed_time)
# time taken: 0.006171464920043945

time taken: 0.006605386734008789


In [ ]:
# fetch file from the bucket

def fetch_files ():
    star_fn = time.time()
    storage_client = storage.Client()
    bucket_name = "development_awarri"
    prefix = "read/english/src-docs"
    source_bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=None)
    for blob in blobs:
        file = blob.name
        blob_file = source_bucket.blob(file)
        with blob_file.open("r") as f:
            text = f.read()
            print(text)
    end_fn = time.time()
    print("fn time:",end_fn-star_fn)
fetch_files()   
